In [1]:
import os, re
import pandas as pd

# os.chdir('/GPUFS/sysu_jhluo_1/wangyh/data/FAH_BLCA/ext_val_dsmil')
# os.getcwd()

'/GPUFS/sysu_jhluo_1/wangyh/data/FAH_BLCA/ext_val_dsmil'

In [2]:
def metrics_recording(description):
    with open(f'{description}/metrics.log') as f:

#     pattern = re.compile(r'lr:(.+),weight:([\d\.]+),fold:(\d+),metric:(\w+),avg_score: ([\d\.]+),auc for TMB-H: ([\d\.]+),f1:([\d\.]+),tn_fp_fn_tp:\((\d+), (\d+), (\d+), (\d+)\)')
        pattern = re.compile(r'lr:(.+),weight:(.+),fold:(.+),metric:(.+),avg_score: (.+),auc for TMB-H: ([\d\.]+),f1:([\d\.]+),tn_fp_fn_tp:\((\d+), (\d+), (\d+), (\d+)\)')
        log = f.read()
        data = []
        for line in log.split('\n'):
            match = pattern.search(line)
    #         print(match.group(9))
            if match:
                data.append([float(match.group(1)), float(match.group(2)), int(match.group(3)), match.group(4), 
                            float(match.group(5)), float(match.group(6)), float(match.group(7)), 
                            int(match.group(8)), int(match.group(9)), int(match.group(10)), int(match.group(11))])

        df = pd.DataFrame(data, columns=['lr', 'weight', 'fold', 'metric', 'avg_score', 'auc', 'f1', 'tn', 'fp', 'fn', 'tp'])
        return df
def metrics_recording_with_dropout(description):
    with open(f'{description}/metrics.log') as f:

#     pattern = re.compile(r'lr:(.+),weight:([\d\.]+),fold:(\d+),metric:(\w+),avg_score: ([\d\.]+),auc for TMB-H: ([\d\.]+),f1:([\d\.]+),tn_fp_fn_tp:\((\d+), (\d+), (\d+), (\d+)\)')
        pattern = re.compile(r'lr:(.+),weight:(.+),dropout_patch:(.+),dropout_node:(.+),fold:(.+),metric:(.+),avg_score: (.+),auc for TMB-H: ([\d\.]+),f1:([\d\.]+),tn_fp_fn_tp:\((\d+), (\d+), (\d+), (\d+)\)')
        log = f.read()
        data = []
        for line in log.split('\n'):
            match = pattern.search(line)
    #         print(match.group(9))
            if match:
                data.append([float(match.group(1)), float(match.group(2)), float(match.group(3)), float(match.group(4)),
                             int(match.group(5)), match.group(6), 
                            float(match.group(7)), float(match.group(8)), float(match.group(9)), 
                            int(match.group(10)), int(match.group(11)), int(match.group(12)), int(match.group(13))])

        df = pd.DataFrame(data, columns=['lr', 'weight','dropout_patch','dropout_node', 'fold', 'metric', 'avg_score', 'auc', 'f1', 'tn', 'fp', 'fn', 'tp'])
        return df
    
def show_topK_metrics(df,k,metrics):
    top_idx = df.sort_values(f'{metrics}', ascending=False).head(k).index
    result = df.loc[top_idx,:]
    print(df[f'{metrics}'].describe())
    return result

def eval_folder(folder,mode='norm'):
    if mode == 'norm':
        df = metrics_recording(folder)
        df = df[~df.duplicated()]
    elif mode == 'dropout':
        df = metrics_recording_with_dropout(folder)
        df = df[~df.duplicated()]
    print(show_topK_metrics(df,10,'auc'))
    print(show_topK_metrics(df,10,'f1'))

In [3]:
eval_folder('10X_5fold_res18_ft_dropout_node',mode='dropout')

count    850.000000
mean       0.518310
std        0.041762
min        0.435065
25%        0.485931
50%        0.524892
75%        0.531385
max        0.638528
Name: auc, dtype: float64
           lr    weight  dropout_patch  dropout_node  fold      metric  \
430  0.000675  0.000301            0.3           0.1     0  best_score   
52   0.000675  0.000301            0.3           0.0     0  best_score   
259  0.000675  0.000301            0.3           0.2     0  best_score   
322  0.000675  0.000301            0.3           0.5     0  best_score   
163  0.000675  0.000301            0.3           0.3     0  best_score   
343  0.000675  0.000301            0.3           0.4     0  best_score   
53   0.000675  0.000301            0.3           0.0     0    best_auc   
431  0.000675  0.000301            0.3           0.1     0    best_auc   
164  0.000675  0.000301            0.3           0.3     0    best_auc   
323  0.000675  0.000301            0.3           0.5     0    best_auc   


In [4]:
df = metrics_recording_with_dropout('10X_5fold_res18_ft_dropout_node')
df = df[~df.duplicated()]
df

,lr,weight,dropout_patch,dropout_node,fold,metric,avg_score,auc,f1,tn,fp,fn,tp
0,0.000675,0.000348,0.0,0.3,0,best_loss,0.584615,0.522727,0.428571,43,1,15,6
1,0.000675,0.000348,0.0,0.3,0,best_score,0.369231,0.551948,0.538462,27,17,7,14
2,0.000675,0.000348,0.0,0.3,0,best_auc,0.353846,0.525974,0.500000,29,15,9,12
3,0.000123,0.000348,0.5,0.5,0,best_loss,0.646154,0.525974,0.428571,43,1,15,6
4,0.000123,0.000348,0.5,0.5,0,best_score,0.369231,0.546537,0.478261,30,14,10,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,0.000123,0.000348,0.0,0.3,3,best_score,0.384615,0.525974,0.538462,27,17,7,14
906,0.000123,0.000301,0.0,0.1,3,best_score,0.338462,0.520563,0.548387,20,24,4,17
907,0.000123,0.000301,0.0,0.3,3,best_score,0.338462,0.520563,0.548387,20,24,4,17
908,0.000123,0.000301,0.0,0.5,3,best_score,0.338462,0.520563,0.548387,20,24,4,17


In [5]:
show_topK_metrics(df,10,'auc')

count    850.000000
mean       0.518310
std        0.041762
min        0.435065
25%        0.485931
50%        0.524892
75%        0.531385
max        0.638528
Name: auc, dtype: float64


,lr,weight,dropout_patch,dropout_node,fold,metric,avg_score,auc,f1,tn,fp,fn,tp
430,0.000675,0.000301,0.3,0.1,0,best_score,0.615385,0.638528,0.500000,35,9,11,10
52,0.000675,0.000301,0.3,0.0,0,best_score,0.615385,0.638528,0.500000,35,9,11,10
259,0.000675,0.000301,0.3,0.2,0,best_score,0.615385,0.638528,0.500000,35,9,11,10
322,0.000675,0.000301,0.3,0.5,0,best_score,0.615385,0.638528,0.500000,35,9,11,10
163,0.000675,0.000301,0.3,0.3,0,best_score,0.615385,0.638528,0.500000,35,9,11,10
343,0.000675,0.000301,0.3,0.4,0,best_score,0.615385,0.638528,0.500000,35,9,11,10
53,0.000675,0.000301,0.3,0.0,0,best_auc,0.476923,0.624459,0.533333,32,12,9,12
431,0.000675,0.000301,0.3,0.1,0,best_auc,0.476923,0.624459,0.533333,32,12,9,12
164,0.000675,0.000301,0.3,0.3,0,best_auc,0.476923,0.624459,0.533333,32,12,9,12
323,0.000675,0.000301,0.3,0.5,0,best_auc,0.476923,0.624459,0.533333,32,12,9,12


In [6]:
show_topK_metrics(df,10,'f1')

count    850.000000
mean       0.439909
std        0.117749
min        0.000000
25%        0.418605
50%        0.464286
75%        0.521739
max        0.588235
Name: f1, dtype: float64


,lr,weight,dropout_patch,dropout_node,fold,metric,avg_score,auc,f1,tn,fp,fn,tp
271,0.000123,0.000348,0.0,0.5,0,best_score,0.369231,0.550866,0.588235,29,15,6,15
427,0.000123,0.000348,0.0,0.1,0,best_score,0.369231,0.550866,0.588235,29,15,6,15
310,0.000123,0.000348,0.0,0.3,0,best_score,0.369231,0.550866,0.588235,29,15,6,15
46,0.000123,0.000348,0.0,0.2,0,best_score,0.369231,0.550866,0.588235,29,15,6,15
13,0.000123,0.000348,0.0,0.4,0,best_score,0.369231,0.550866,0.588235,29,15,6,15
361,0.000675,0.000301,0.4,0.2,0,best_score,0.446154,0.589827,0.584615,19,25,2,19
376,0.000675,0.000301,0.4,0.3,0,best_score,0.446154,0.589827,0.584615,19,25,2,19
388,0.000675,0.000301,0.4,0.4,0,best_score,0.446154,0.589827,0.584615,19,25,2,19
214,0.000675,0.000301,0.4,0.5,0,best_score,0.446154,0.589827,0.584615,19,25,2,19
196,0.000675,0.000301,0.4,0.1,0,best_score,0.446154,0.589827,0.584615,19,25,2,19


In [1]:
import pickle
with open('best_loss_10X_FAH_pred.pkl','rb') as f:
    int_pred_loss = pickle.load(f)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [3]:
import pandas as pd
df = pd.DataFrame(int_pred_loss).T
df = df.reset_index()
df.columns=['slide_id','pred','logits','label']
# df['slide_id'] = df['slide_id'].map(slideid_uuid_dict)
df.to_csv('TCGA_pred_with_logits.csv')

In [4]:
with open('/GPUFS/sysu_jhluo_1/wangyh/data/TCGA_bladder_threshold_80/best_ckpt/best_loss.pkl','rb') as f:
    threshold = pickle.load(f)

In [5]:
threshold

[0.8601421, 0.20119178]